추가할 내용
1. 챗봇 페르소나 지정해주기
2. db 연결해서 내 정보 기억하기
3. api key 입력받아서 사용하기

In [1]:
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langgraph.graph import START, StateGraph, END
from IPython.display import Image, display
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from utils.util import trace_function, google_search_scrape, extract_content, set_env, PINK, RESET, initialize_browser
from configs.config import prompt_config

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
ShortTermMemory = MemorySaver()
set_env()
llm = ChatOpenAI(model="gpt-4o")

class State(MessagesState):
    is_search: str

In [3]:
@trace_function(enable_print=True, only_node=True)
def Node_search(state: State):
    query = state['messages'][-1].content # TODO humanmessage 인지 체크필요
    prompt = prompt_config.generate_search_info.format(query=query)
    search_info = llm.invoke(prompt).content
    print(f"{PINK}\n{search_info}\n{RESET}")
    
    SEARCH_RETRY_COUNT = 5
    SEARCH_RESULT_COUNT = 4
    SEARCH_MINIMUM_RESULT = 1
    for _ in range(SEARCH_RETRY_COUNT):
        results = google_search_scrape(search_info, num_results=SEARCH_RESULT_COUNT)
        if len(results) >= SEARCH_MINIMUM_RESULT:
            break
    # TODO 결과없을때 처리필요
    context = ''
    suffix = ''
    for idx, result in enumerate(results):
        link = result.get("link")
        desc, detailed_content = extract_content(link)
        if "Enable JavaScript and cookies" in detailed_content:
            continue
        context += f"제목 : {result.get('title')}\n링크 : {link}\n설명 : {desc}\n내용 : {detailed_content}\n\n"    
        suffix += f"""
📌 참고내용 [{idx+1}]
제목 : {result.get('title')}
링크 : {link}
설명 : {desc}
"""
    prompt = prompt_config.answer_with_context.format(context=context, query=query)
    answer = llm.invoke(prompt).content + "\n" + suffix
    return {"messages": [AIMessage(content=answer)]}

@trace_function(enable_print=True, only_node=True)
def decide_search(state: State):
    if "YES" in state["is_search"]:
        return "Node_search"
    else:
        return END

@trace_function(enable_print=True, only_node=True)
def Node_decide_search(state: State):
    return {"is_search": [llm.invoke([SystemMessage(content=prompt_config.decide_search_prompt)] + state["messages"])][0].content.upper()}

builder = StateGraph(State)
builder.add_node("Node_decide_search", Node_decide_search)
builder.add_node("Node_search", Node_search)
builder.add_edge(START, "Node_decide_search")
builder.add_conditional_edges("Node_decide_search", decide_search)
builder.add_edge("Node_search", END)
graph = builder.compile(checkpointer=ShortTermMemory)

In [4]:
config = {"configurable": {"thread_id": "initial_chat",
                           "user_id": "changwoo2"}}

graph.invoke({"messages":"최근 테슬라 주가에 대해서 알려줘"}, config=config)


🚀 Passing Through [Node_decide_search] ..

🚀 Passing Through [decide_search] ..

🚀 Passing Through [Node_search] ..

"2025년 1월 테슬라 주가 동향"



Fetching pages: 100%|##########| 1/1 [00:00<00:00,  7.40it/s]


{'messages': [HumanMessage(content='최근 테슬라 주가에 대해서 알려줘', additional_kwargs={}, response_metadata={}, id='b536c298-b7a9-4b1a-bad3-13bd1f98e99f'),
  AIMessage(content='2025년 1월 3일의 금융시장 동향에 따르면, 미국 증시는 5일 연속 하락하면서 주요 지수들이 소폭 하락 마감되었습니다. S&P500과 나스닥 100 지수는 각각 0.2% 하락했으며, 이로 인해 시가총액이 1조 달러 이상 감소했습니다. 또한, 테슬라는 10년 만에 처음으로 연간 판매량 감소를 기록하며 주가가 하락했습니다. 미국채 10년물 금리는 소폭 하락했으며, 달러는 강세를 보였습니다. 유럽 경제에 대한 우려로 인해 유로화와 파운드화는 각각 최저치를 기록했습니다. 이러한 상황은 미국의 금리 인상 지속과 중국 경제의 불확실성 등이 영향을 미친 것으로 보입니다. 투자자들은 주가의 단기 변동성보다는 장기적인 경제 상황을 주시하며 신중한 접근이 필요합니다.\n\n📌 참고내용 [1]\n제목 : 2025년 1월 3일 금융시장 동향\n링크 : https://kbthink.com/main/economy/trend/daily-market-trend/2025/daily-market-trend-2025-01-03.html\n설명 : 지난 밤 미국 증시 주요 지수들은 소폭 하락 마감. 주요 지수들이 5거래일 연속 하락하면서 시가총액이 1조 달러 이상 감소. SP500과 나스닥 100 지수는 모두 0.2% 하락 \n\n📌 참고내용 [2]\n제목 : 2025년 01월 04일 테슬라 최신 동향\n링크 : https://www.evdbs.com/info-table/tesla-info/tesla-news-2025-analysis/\n설명 : 제공된 뉴스와 주식 데이터를 바탕으로 테슬라 주식의 현황과 향후 전망을 분석해 보겠습니다. 주식 데이터의 이동평균과 RSI 값이 제공되지 않아 기술적 분석은

In [2]:
search_info = "2025년 1월 테슬라 주가 동향"
results = google_search_scrape(search_info, num_results=5)
context = ''
suffix = ''
for idx, result in enumerate(results):
    link = result.get("link")
    desc, detailed_content = extract_content(link)
    context += f"제목 : {result.get('title')}\n링크 : {link}\n설명 : {desc}\n내용 : {detailed_content}\n\n"    
    suffix += f"""
📌 참고내용 [{idx+1}]
제목 : {result.get('title')}
링크 : {link}
설명 : {desc}
"""

len(results)

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 11.39it/s]


4

In [7]:
link = "https://kr.investing.com/equities/tesla-motors-historical-data"

In [11]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
loader = AsyncHtmlLoader(link)
docs = loader.load()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  5.22it/s]


In [13]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [17]:
docs_transformed[0].page_content

'Enable JavaScript and cookies to continue\n\n'